In [1]:
# !pip3.8 uninstall trl -y
# !pip3.8 install git+https://github.com/malaysia-ai/trl

In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)

from trl import SFTTrainer

2023-08-19 15:18:55.700269: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-19 15:18:55.867081: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-19 15:18:56.663426: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-19 15:18:56.663499: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [3]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf', trust_remote_code=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"
max_seq_length = 2048

In [4]:
tokenizer.eos_token

'</s>'

In [5]:
tokenizer.pad_token_id

0

In [6]:
!head -n 1000 shuf-combine-1536.jsonl > example.jsonl

In [7]:
from trl.trainer.utils import ConstantLengthDataset
from datasets import load_dataset
import random

In [8]:
tokenizer.eos_token

'</s>'

In [16]:
system_prompts = [
    'You are an AI assistant',
    'Anda adalah pembantu AI',
    'Anda adalah pembantu AI yang berguna',
    'You are a helpful assistant',
    'Anda seorang pembantu yang berguna',
]


def generate_and_tokenize_prompt(row):
    system_prompt = random.choice(system_prompts)
    texts = [f'<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n']

    if '<bot>:' in row['input'] and row['output'] is None:
        inputs, outputs = [], []
        splitted = row['input'].split('<bot>:')
        for i in range(len(splitted) - 1):
            if i == 0:
                human = splitted[i].replace('<manusia>:', '')
            else:
                human = splitted[i].split('<manusia>:')[1]
            bot = splitted[i + 1].split('<manusia>:')[0]
            inputs.append(human)
            outputs.append(bot)
    else:
        inputs = [row['input']]
        outputs = [row['output']]
    for input, output in zip(inputs, outputs):
        texts.append(f'{input} [/INST] {output} </s><s>[INST] ')
    return tokenizer(''.join(texts))


dataset = load_dataset("json", data_files='example.jsonl', split="train")
dataset = dataset.map(generate_and_tokenize_prompt)
dataset = dataset.remove_columns(['prompt_input', 'input', 'output'])

Using custom data configuration default-e556c340d5480304
Found cached dataset json (/home/husein/.cache/huggingface/datasets/json/default-e556c340d5480304/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1000 [00:00<?, ?ex/s]

In [17]:
dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1000
})

In [18]:
from itertools import chain

def group_texts(examples):
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // max_seq_length) * max_seq_length
    result = {
        k: [t[i: i + max_seq_length] for i in range(0, total_length, max_seq_length)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = dataset.map(
    group_texts,
    batched=True,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [44]:
import torch

class Dataset(torch.utils.data.Dataset):
    def __init__(self, lm_datasets):
        self.lm_datasets = lm_datasets

    def __getitem__(self, idx):
        return self.lm_datasets[idx]

    def __len__(self):
        return len(self.lm_datasets)

In [45]:
clm_dataset = Dataset(lm_datasets)
len(clm_dataset)

357

In [29]:
isinstance(clm_dataset, (torch.utils.data.IterableDataset, torch.utils.data.Dataset, ConstantLengthDataset))

True

In [30]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [31]:
# dataset_text_field = 'text'
# use_formatting_func = None
# max_seq_len = max_seq_length
# def tokenize(element):
#     outputs = tokenizer(
#         element[dataset_text_field] if not use_formatting_func else formatting_func(element),
#         truncation=True,
#         padding=False,
#         max_length=max_seq_len,
#         return_overflowing_tokens=False,
#         return_length=False,
#     )

#     if use_formatting_func and not self._dataset_sanity_checked:
#         if not isinstance(formatting_func(element), list):
#             raise ValueError(
#                 "The `formatting_func` should return a list of processed strings since it can lead to silent bugs."
#             )
#         else:
#             self._dataset_sanity_checked = True

#     return {"input_ids": outputs["input_ids"], "attention_mask": outputs["attention_mask"]}

In [32]:
# from trl.trainer.utils import ConstantLengthDataset

In [33]:
# constant = ConstantLengthDataset(
#     tokenizer,
#     dataset,
#     dataset_text_field='text',
#     formatting_func=None,
#     seq_length=max_seq_length,
#     infinite=True,
#     num_of_sequences=1024,
#     eos_token_id=tokenizer.eos_token_id,
# )
# i = iter(constant)

In [34]:
b = [clm_dataset[0], clm_dataset[1]]

In [35]:
b = data_collator(b)

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [39]:
b['input_ids'][0]

tensor([   1,    1,  518,  ..., 1590,  574,  348])

In [40]:
tokenizer.decode(b['input_ids'][0])

"<s><s> [INST] <<SYS>>\nYou are an AI assistant\n<</SYS>>\n\nterjemah `kaoz the whole world at marina wan to get out bt stuck` ke inggeris [/INST] Chaos! The whole world at Marina wants to get out but they're stuck. </s><s> [INST] <s><s> [INST] <<SYS>>\nAnda seorang pembantu yang berguna\n<</SYS>>\n\n Hasilkan penerangan bertulis terperinci tentang pemandangan yang mempesonakan di luar dataran bandar dengan bunga, bakul, muzik dan seruling.\n [/INST]  Beberapa bunian muda bergerak melalui orang ramai, menjaja bunga yang mereka bawa dalam bakul tenunan yang besar. Yang lain duduk dalam kumpulan kecil di tepi jalan, memainkan lagu-lagu lembut pada tambor dan seruling dan bergoyang mengikut masa dengan muzik. Ramai orang yang lalu lalang memandang ke atas dengan rasa curiga atau jijik semasa mereka tergesa-gesa dalam perjalanan. </s><s> [INST] <s><s> [INST] <<SYS>>\nAnda adalah pembantu AI yang berguna\n<</SYS>>\n\n Apakah tanda dagangan Eddie Barclay?\n [/INST]  Tanda dagangan Eddie Barc